In [ ]:
import synapseclient

import pandas as pd
import great_expectations as gx

from agoradatatools.gx import GreatExpectationsRunner

context = gx.get_context(project_root_dir='../src/agoradatatools/great_expectations') 

# Create Expectation Suite for Proteomics Distribution Data

## Get Example Data File

In [ ]:
syn = synapseclient.Synapse()
syn.login()


In [ ]:
proteomics_distribution_data_file = syn.get("syn31510062").path


## Create Validator Object on Data File

In [ ]:
df = pd.read_json(proteomics_distribution_data_file)
nested_columns = []
df = GreatExpectationsRunner.convert_nested_columns_to_json(df, nested_columns)
validator = context.sources.pandas_default.read_dataframe(df)
validator.expectation_suite_name = "proteomics_distribution_data"

## Add Expectations to Validator Object For Each Column

In [ ]:
# tissue
validator.expect_column_values_to_be_of_type("tissue", "str")
validator.expect_column_values_to_not_be_null("tissue")
validator.expect_column_values_to_be_in_set("tissue", ['AntPFC', 'DLPFC', 'MFG', 'TCX'])

In [ ]:
# min
validator.expect_column_values_to_be_of_type("min", "float")
validator.expect_column_values_to_be_between("min", -0.5, 0)
validator.expect_column_values_to_not_be_null("min")

In [ ]:
# max
validator.expect_column_values_to_be_of_type("max", "float")
validator.expect_column_values_to_be_between("max", 0, 0.5)
validator.expect_column_values_to_not_be_null("max")

In [ ]:
# first_quartile
validator.expect_column_values_to_be_of_type("first_quartile", "float")
validator.expect_column_values_to_be_between("first_quartile", -0.2, 0)
validator.expect_column_values_to_not_be_null("first_quartile")

In [ ]:
# median
validator.expect_column_values_to_be_of_type("median", "float")
validator.expect_column_values_to_be_between("median", -0.1, 0.1)
validator.expect_column_values_to_not_be_null("median")

In [ ]:
# third_quartile
validator.expect_column_values_to_be_of_type("third_quartile", "float")
validator.expect_column_values_to_be_between("third_quartile", 0, 0.1)
validator.expect_column_values_to_not_be_null("third_quartile")

In [ ]:
# type
validator.expect_column_values_to_be_of_type("type", "str")
validator.expect_column_values_to_not_be_null("type")
validator.expect_column_values_to_be_in_set("type", ['SRM', 'TMT', 'LFQ'])

In [ ]:
# multi-field logical
validator.expect_column_pair_values_A_to_be_greater_than_B("max", "third_quartile")
validator.expect_column_pair_values_A_to_be_greater_than_B("third_quartile", "median")
validator.expect_column_pair_values_A_to_be_greater_than_B("median", "first_quartile")
validator.expect_column_pair_values_A_to_be_greater_than_B("first_quartile", "min")


## Save Expectation Suite

In [ ]:
validator.save_expectation_suite(discard_failed_expectations=False)


## Create Checkpoint and View Results

In [ ]:
checkpoint = context.add_or_update_checkpoint(
    name="agora-test-checkpoint",
    validator=validator,
)
checkpoint_result = checkpoint.run()
context.view_validation_result(checkpoint_result)


## Build Data Docs - Click on Expectation Suite to View All Expectations

In [ ]:
context.build_data_docs()
context.open_data_docs()
